# Store JSON Like a Pro - PostgreSQL JSONB Full Guide (with Python)

**What you'll learn**
- JSON vs JSONB in PostgreSQL
- How to store, query, update JSONB
- Useful operators & functions (`->`, `->>`, `@>`, ...)
- Indexing JSONB for performance (GIN, expression, generated columns)
- Python examples (psycopg2 + pandas)

## Prerequisites

- PostgreSQL installed and running.
- Python environment (>=3.8).
- Jupyter Notebook / JupyterLab.

We will use:
- `psycopg2-binary` for DB connection
- `pandas` for convenience in displaying results
- `python-dotenv` (optional) to load DB creds from `.env`

In [ ]:
# Run this cell to install required packages (if not already installed)
!pip install psycopg2-binary pandas python-dotenv tqdm

## 1. Setup (DB Connection & Imports)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import psycopg2
import psycopg2.extras
import pandas as pd
from tqdm import tqdm

# Load .env if present
print("Environment Variables Loaded? ", load_dotenv())

DB_CONFIG = {
    "host": os.getenv("PG_HOST", "localhost"),
    "port": os.getenv("PG_PORT", "5432"),
    "dbname": os.getenv("PG_DATABASE", "postgres"),
    "user": os.getenv("PG_USER", "postgres"),
    "password": os.getenv("PG_PASS", "postgres"),
}

Environment Variables Loaded?  True


### 1.1 Test DB Connection

In [3]:
def get_connection():
    connection = psycopg2.connect(**DB_CONFIG)
    connection.autocommit=True
    cursor = connection.cursor()

    return connection, cursor
    

# Quick connection test
try:
    connection, cursor = get_connection()
    print("Connected to PostgreSQL")
except Exception as e:
    print("Connection failed:", e)
finally:
    cursor.close()
    connection.close()

Connected to PostgreSQL


## 2. Sample Data

We will add 10 sample articles stored as JSONB. Each JSON will have fields like:
- title
- author (object with name / id)
- tags (array)
- body (text)
- metrics (nested object with views, likes)

In [4]:
sample_docs = [
    {
        "title": "Intro to Transformers",
        "author": {"id": "u1", "name": "Asha"},
        "tags": ["ai", "nlp", "transformers"],
        "body": "Transformers are a type of model that use self-attention...",
        "metrics": {"views": 1200, "likes": 150}
    },
    {
        "title": "Getting Started with PGVector",
        "author": {"id": "u2", "name": "Sunny"},
        "tags": ["postgres", "pgvector", "vectors"],
        "body": "PGVector adds a vector type to Postgres enabling embeddings...",
        "metrics": {"views": 800, "likes": 90}
    },
    {
        "title": "Fine-tuning Tips for LLMs",
        "author": {"id": "u3", "name": "Maya"},
        "tags": ["llm", "fine-tuning", "ml"],
        "body": "Fine-tuning can help models specialize; remember to curate data...",
        "metrics": {"views": 500, "likes": 45}
    },
    {
        "title": "Practical JSONB Patterns",
        "author": {"id": "u4", "name": "Ravi"},
        "tags": ["postgres", "jsonb", "database"],
        "body": "JSONB is a binary JSON format in Postgres that's efficient to query...",
        "metrics": {"views": 300, "likes": 25}
    },
    {
        "title": "Vector Search vs Keyword Search",
        "author": {"id": "u2", "name": "Sunny"},
        "tags": ["search", "vectors", "nlp"],
        "body": "Vector search uses embeddings to find semantic matches; keyword search uses tokens...",
        "metrics": {"views": 1000, "likes": 120}
    },
    {
        "title": "Monitoring ML Models",
        "author": {"id": "u5", "name": "Priya"},
        "tags": ["ml", "monitoring", "ops"],
        "body": "Monitoring models in production is essential for performance and safety...",
        "metrics": {"views": 200, "likes": 20}
    },
    {
        "title": "Composable Retrieval Pipelines",
        "author": {"id": "u3", "name": "Maya"},
        "tags": ["rag", "retrieval", "llm"],
        "body": "RAG combines retrieval with generative models for grounded answers...",
        "metrics": {"views": 650, "likes": 60}
    },
    {
        "title": "Building a Semantic Cache",
        "author": {"id": "u1", "name": "Asha"},
        "tags": ["cache", "semantic", "performance"],
        "body": "A semantic cache stores embeddings of recent queries and results...",
        "metrics": {"views": 90, "likes": 5}
    },
    {
        "title": "Scaling Postgres for AI",
        "author": {"id": "u4", "name": "Ravi"},
        "tags": ["postgres", "scaling", "infrastructure"],
        "body": "Scale Postgres with sharding and read replicas; choose indexes wisely...",
        "metrics": {"views": 420, "likes": 30}
    },
    {
        "title": "Ethics of LLMs",
        "author": {"id": "u5", "name": "Priya"},
        "tags": ["ethics", "ai", "policy"],
        "body": "Ethical considerations include bias, transparency, and safety...",
        "metrics": {"views": 230, "likes": 28}
    }
]

## 3. Insert Data into DB

### 3.1 Create Table

In [5]:
create_sql = """
DROP TABLE IF EXISTS articles;
CREATE TABLE articles (
    id SERIAL PRIMARY KEY,
    data JSONB,
    created_at TIMESTAMPTZ DEFAULT NOW()
);
"""


try:
    connection, cursor =  get_connection()
    print("Connection Successful")
    
    cursor.execute(create_sql)
    print("Table `articles` created.")
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

Connection Successful
Table `articles` created.


### 3.2 Insert Data into Table

In [7]:
insert_sql = "INSERT INTO articles (data) VALUES (%s);"

try:
    connection, cursor =  get_connection()
    print("Connection Successful")
    
    for doc in sample_docs:
        cursor.execute(insert_sql, (psycopg2.extras.Json(doc),))
        
    print("Inserted sample JSONB documents.")
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

Connection Successful
Inserted sample JSONB documents.


### 4. JSON vs JSONB

* `json` stores text: preserves exact input, slower to query because parsing is needed.
    * **Storage:** Stores the JSON data as a plain text string, preserving the exact input format, including whitespace, key order, and duplicate keys.
    * **Processing:** Requires re-parsing the text string every time it is accessed or queried, leading to slower processing for read-heavy operations.
    * **Use Cases:** Suitable when the exact textual representation of the JSON is critical, such as for logging or when preserving specific formatting is a requirement.
* `jsonb` stores a binary representation: faster to query, supports indexing, does not preserve original whitespace/order.
    * **Storage:** Stores the JSON data in a decomposed binary format. This format optimizes for efficient storage and faster processing. It removes unnecessary whitespace, does not guarantee key order, and only stores the last value for duplicate keys.
    * **Processing:** Stores the data in a pre-parsed binary format, eliminating the need for re-parsing during queries and manipulation, resulting in significantly faster performance for read and write operations.
    * **Indexing:** Supports advanced indexing capabilities (e.g., GIN indexes), which can drastically improve query performance on large datasets.
    * **Use Cases:** Generally the preferred choice for most applications due to its performance benefits, efficient storage, and indexing capabilities, especially when querying and manipulating JSON data frequently.


By default prefer `jsonb` unless you need to preserve original formatting. Our `articles.data` uses `jsonb`.

Summary of Key Differences:

| **Feature**        | **json**                              | **jsonb**                                         |
|--------------------|----------------------------------------|--------------------------------------------------|
| **Storage**        | Plain text string                     | Decomposed binary format                         |
| **Formatting**     | Preserves exact input format           | Removes whitespace, normalizes data              |
| **Key Order**      | Preserves key order                    | Does not guarantee key order                     |
| **Duplicate Keys** | Preserves all duplicate keys           | Stores only the last value for duplicates        |
| **Parsing**        | Requires re-parsing on each access     | Pre-parsed, faster access                        |
| **Performance**    | Slower for querying and manipulation   | Faster for querying and manipulation             |
| **Indexing**       | No direct indexing support             | Supports advanced indexing (e.g., GIN)           |



## 5. Basic JSONB Operators & Extraction

#### Useful operators

- `->` : get JSON object field (returns JSON)
- `->>`: get JSON object field as text
- `#>`: get JSON at path (returns JSON)
- `#>>`: get JSON at path as text
- `@>`: containment (does the left JSON contain right JSON)
- `?` `?&` `?|` : key/keys/any key operators (for JSONB)


In [18]:
try:
    connection, cursor =  get_connection()
    
    # 1. Extract title and author name
    cursor.execute("SELECT id, data->>'title' AS title, data->'author'->>'name' AS author FROM articles LIMIT 5;")
    print(cursor.fetchall())

    print("".join(['=']*100))
    
    # 2. Filter where author name = 'Sunny'
    cursor.execute("SELECT id, data->>'title' AS title FROM articles WHERE data->'author'->>'name' = 'Sunny';")
    print("\nBy author='Sunny':", cursor.fetchall())

    print("".join(['=']*100))
    
    # 3. Containment: find docs containing tag 'jsonb'
    cursor.execute("SELECT id, data->>'title' FROM articles WHERE data->'tags' @> '\"jsonb\"'::jsonb;")
    print("\nContain tag 'jsonb':", cursor.fetchall())

    print("".join(['=']*100))
    
    # 4. Array contains any of several tags
    cursor.execute("SELECT id, data->>'title' FROM articles WHERE data->'tags' ?| array['nlp','rag'];")
    print("\nHas any of ['nlp','rag']:", cursor.fetchall())

except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

[(4, 'Practical JSONB Patterns', 'Ravi'), (5, 'Vector Search vs Keyword Search', 'Sunny'), (6, 'Monitoring ML Models', 'Priya'), (7, 'Composable Retrieval Pipelines', 'Maya'), (8, 'Building a Semantic Cache', 'Asha')]

By author='Sunny': [(5, 'Vector Search vs Keyword Search'), (2, 'Getting Started with PGVector')]

Contain tag 'jsonb': [(4, 'Practical JSONB Patterns')]

Has any of ['nlp','rag']: [(5, 'Vector Search vs Keyword Search'), (7, 'Composable Retrieval Pipelines'), (1, 'Intro to Transformers')]


## 6. Querying Nested Arrays / Exploding Arrays

#### Expand arrays with `jsonb_array_elements` to search inside each tag or nested object


In [14]:
try:
    connection, cursor =  get_connection()
    
    query = """
    SELECT a.id, a.data->>'title' AS title, tag.value AS tag
    FROM articles a, jsonb_array_elements(a.data->'tags') AS tag(value)
    ORDER BY a.id;
    """
    df = pd.read_sql(query, connection)

except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()
    
df.head(20)

/var/folders/_p/y8g5n0tx6ls71vbzlylmg2900000gn/T/ipykernel_2155/2271303808.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,id,title,tag
0,1,Intro to Transformers,ai
1,1,Intro to Transformers,nlp
2,1,Intro to Transformers,transformers
3,2,Getting Started with PGVector,postgres
4,2,Getting Started with PGVector,pgvector
5,2,Getting Started with PGVector,vectors
6,3,Fine-tuning Tips for LLMs,llm
7,3,Fine-tuning Tips for LLMs,fine-tuning
8,3,Fine-tuning Tips for LLMs,ml
9,4,Practical JSONB Patterns,postgres


## 7. Updating JSONB fields

#### Common updates

* `jsonb_set` to update or create a field (returns new JSONB)
* concatenation `||` to merge two jsonb objects
* delete key `-` or path deletion using `#-`

**Note:** jsonb_set requires path as text array **'{a,b}'** and value as jsonb. The **false or true** argument controls creation if missing.

In [19]:
try:
    connection, cursor =  get_connection()

    # 1. Add a new field `published: true` to doc id=1
    cursor.execute("UPDATE articles SET data = jsonb_set(data, '{published}', 'true'::jsonb, true) WHERE id = 1 RETURNING data;")
    print("Added published to id=1:", cursor.fetchone()[0])

    print("".join(['=']*100))
    
    # 2. Increment views inside metrics (read current, modify in Python, then set)
    cursor.execute("SELECT id, data->'metrics'->>'views' AS views FROM articles WHERE id = 2;")
    row = cursor.fetchone()
    print("Before views:", row)
    # do update: add 100 views
    cursor.execute("""
    UPDATE articles
    SET data = jsonb_set(data, '{metrics,views}', to_jsonb((data->'metrics'->>'views')::int + 100), false)
    WHERE id = 2
    RETURNING data->'metrics'->>'views';
    """)
    print("After views:", cursor.fetchone()[0])

    print("".join(['=']*100))
    
    # 3. Remove key 'body' from id=3 using - operator on top-level (returns jsonb)
    cursor.execute("UPDATE articles SET data = data - 'body' WHERE id = 3 RETURNING data;")
    print("Removed body from id=3 (data):", cursor.fetchone()[0])

except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

Added published to id=1: {'body': 'Transformers are a type of model that use self-attention...', 'tags': ['ai', 'nlp', 'transformers'], 'title': 'Intro to Transformers', 'author': {'id': 'u1', 'name': 'Asha'}, 'metrics': {'likes': 150, 'views': 1200}, 'published': True}
Before views: (2, '900')
After views: 1000
Removed body from id=3 (data): {'tags': ['llm', 'fine-tuning', 'ml'], 'title': 'Fine-tuning Tips for LLMs', 'author': {'id': 'u3', 'name': 'Maya'}, 'metrics': {'likes': 45, 'views': 500}}


## 8. Indexing JSONB

#### Indexing strategies (intro-level)
- **GIN index on jsonb column** – great for containment queries (`@>`).
- **GIN jsonb_path_ops** – more compact for containment-only use (older PG versions).
- **Expression index / generated column** – extract frequently queried scalar and index it (fast equality).
- **btree index on expression** – index `(data->>'author')` for equality filtering.


In [22]:
try:
    connection, cursor =  get_connection()

    # 1. GIN index for containment queries
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_articles_data_gin ON articles USING gin (data);")
    
    # 2. Expression index for author name (btree for equality)
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_articles_author ON articles ((data->'author'->>'name'));")
    
    
    # 3. Example: generated column (Postgres 12+) + index — create new table example
    # We'll add a generated column 'author_name' to illustrate (requires ALTER TABLE support)    
    cursor.execute("ALTER TABLE articles ADD COLUMN IF NOT EXISTS author_name TEXT GENERATED ALWAYS AS (data->'author'->>'name') STORED;")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_articles_author_name ON articles (author_name);")
    print("Created generated column author_name and index.")
    print("Indexes created (GIN + expression).")
except Exception as e:
    print("Could not add generated column (check PG version):", e)
finally:
    cursor.close()
    connection.close()

Created generated column author_name and index.
Indexes created (GIN + expression).


#### Use `EXPLAIN ANALYZE` to see whether indexes are used.
Show a containment query plan before/after index creation.


In [25]:
try:
    connection, cursor =  get_connection()

    cursor.execute("""EXPLAIN ANALYZE
    SELECT id FROM articles WHERE data @> '{\"tags\": [\"postgres\"]}'::jsonb;
    """)
    plan = "\n".join(r[0] for r in cursor.fetchall())
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()
    
print(plan)

Seq Scan on articles  (cost=0.00..1.12 rows=1 width=4) (actual time=0.009..0.013 rows=3.00 loops=1)
  Filter: (data @> '{"tags": ["postgres"]}'::jsonb)
  Rows Removed by Filter: 7
  Buffers: shared hit=1
Planning:
  Buffers: shared hit=117
Planning Time: 0.544 ms
Execution Time: 0.055 ms


## 9. Advanced: `jsonb_path_query`/ SQL/JSON path

#### Postgres supports SQL/JSON path queries (`jsonb_path_exists`, `jsonb_path_query`) for complex patterns

This is powerful for nested conditions without unnesting.


In [29]:
try:
    connection, cursor =  get_connection()
    
    # Example: find docs where metrics.views > 500 using jsonb_path_exists
    cursor.execute("""
    SELECT id, data->>'title' AS title
    FROM articles
    WHERE jsonb_path_exists(data, '$.metrics ? (@.views > 500)')
    """)
    print("jsonb_path_exists results:", cursor.fetchall())

    print("".join(['=']*100))
    
    # jsonb_path_query example: extract nested nodes
    cursor.execute("""
    SELECT jsonb_path_query(data, '$.metrics') FROM articles LIMIT 5;
    """)
    print("jsonb_path_query sample:", cursor.fetchall()[:5])
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

jsonb_path_exists results: [(5, 'Vector Search vs Keyword Search'), (7, 'Composable Retrieval Pipelines'), (1, 'Intro to Transformers'), (2, 'Getting Started with PGVector')]
jsonb_path_query sample: [({'likes': 25, 'views': 300},), ({'likes': 120, 'views': 1000},), ({'likes': 20, 'views': 200},), ({'likes': 60, 'views': 650},), ({'likes': 5, 'views': 90},)]


## 10. Using Python: read & write with pandas

In [31]:
try:
    connection, cursor =  get_connection()
    
    df = pd.read_sql("SELECT id, data FROM articles ORDER BY id", connection)
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

# Convert JSONB column to python dict for display
df['data'] = df['data'].apply(lambda x: x)  # psycopg2 returns dict-like already
df.head(5)

/var/folders/_p/y8g5n0tx6ls71vbzlylmg2900000gn/T/ipykernel_2155/1161511468.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT id, data FROM articles ORDER BY id", connection)


,id,data
0,1,{'body': 'Transformers are a type of model tha...
1,2,{'body': 'PGVector adds a vector type to Postg...
2,3,"{'tags': ['llm', 'fine-tuning', 'ml'], 'title'..."
3,4,{'body': 'JSONB is a binary JSON format in Pos...
4,5,{'body': 'Vector search uses embeddings to fin...


In [33]:
# Example: update a row from Python (modify dict then push back)
try:
    connection, cursor =  get_connection()
    
    cursor.execute("SELECT id, data FROM articles WHERE id = 4;")
    row = cursor.fetchone()
    print("Before:", row[1])
    
    d = row[1]
    d['summary'] = "Quick primer on JSONB patterns"
    cursor.execute("UPDATE articles SET data = %s WHERE id = %s;", (psycopg2.extras.Json(d), row[0]))

    print("".join(['=']*100))
    
    cursor.execute("SELECT id, data FROM articles WHERE id = 4;")
    print("After:", cursor.fetchone()[1])
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

Before: {'body': "JSONB is a binary JSON format in Postgres that's efficient to query...", 'tags': ['postgres', 'jsonb', 'database'], 'title': 'Practical JSONB Patterns', 'author': {'id': 'u4', 'name': 'Ravi'}, 'metrics': {'likes': 25, 'views': 300}}
After: {'body': "JSONB is a binary JSON format in Postgres that's efficient to query...", 'tags': ['postgres', 'jsonb', 'database'], 'title': 'Practical JSONB Patterns', 'author': {'id': 'u4', 'name': 'Ravi'}, 'metrics': {'likes': 25, 'views': 300}, 'summary': 'Quick primer on JSONB patterns'}


## 11. Full-text search + JSONB

#### Combining full-text search with JSONB

You may want to search text inside JSONB (body) using PostgreSQL full-text search (tsvector).
Common pattern: add a generated `tsvector` column built from `(data->>'body')` and create a GIN index on it.


In [34]:
try:
    connection, cursor =  get_connection()
    
    # Add tsvector column
    cursor.execute("ALTER TABLE articles ADD COLUMN IF NOT EXISTS body_tsv tsvector GENERATED ALWAYS AS (to_tsvector('english', coalesce(data->>'body',''))) STORED;")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_articles_body_tsv ON articles USING gin (body_tsv);")
    
    print("Added full-text tsvector and index.")
except Exception as e:
    print(e)
finally:
    cursor.close()
    connection.close()

Added full-text tsvector and index.


## 12. Practical Patterns & Tips

- Prefer `jsonb` for storage in Postgres unless you need exact whitespace/ordering.
- Use **GIN** indexes for containment (`@>`) queries.
- Extract frequently queried scalar fields into **generated columns** (or materialized columns) and add btree indexes for equality filters.
- Use `jsonb_array_elements()` to unnest arrays for joins or aggregates.
- Use `jsonb_set` and `-` operator for safe updates; for complex updates extract, mutate in Python and write back via parameterized queries.
- Combine full-text search (tsvector) with semantic search or JSONB for hybrid search features.
- Always `EXPLAIN ANALYZE` when tuning queries on production-sized datasets.


## 13. Exercises for the viewer

### Exercises
1. Add a `category` field inside `data` for some rows and write a query to list titles by category.
2. Create a GIN index on `data->'tags'` and measure query performance for tag lookups.
3. Write a function / stored procedure that increments `metrics.views` safely and test it concurrently.
4. Build a small FastAPI endpoint that returns articles filtered by tag + full-text search.
5. Convert certain keys (like author.name) into a separate table and show how to join with JSONB table.

## 14. Summary

## Wrap-up

In this notebook we:
- Created a `jsonb` column and inserted realistic documents
- Used extraction operators (`->`, `->>`, `@>`) and array functions
- Demonstrated updates (`jsonb_set`, `-`), merging (`||`), and deletes
- Introduced indexing strategies (GIN + expression indexes + generated columns)
- Touched advanced features: SQL/JSON path, full-text search over JSONB, and Python integration

Next tutorial ideas:
- Deep dive on JSONB indexing & benchmarking with large datasets
- Hybrid search: combine JSONB metadata filters + semantic vector search (PGVector)
